In [5]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import json

from pythainlp.corpus import thai_stopwords

In [ ]:
with open('dataset/TripAdvisor_reviews_data.json', 'r') as file:
    reviews_data = json.load(file)
reviews_data

In [12]:
normalize_reviews_data = pd.DataFrame()

for location_id, reviews in reviews_data[0].items():
    print(f"Location ID: {location_id}")
    normalize_reviews_data.append([""])
    for review in reviews:
        print("Review ID:", review['id'])
        print("Username:", review['user']['username'])
        print("Review Text:", review['text'])
        print("Rating:", review['rating'])
        print("Published Date:", review['published_date'])
        print("URL:", review['url'])
        print("-" * 50)

Location ID: 2209612
Review ID: 922231809
Username: maxzerk
Review Text: เป็นวัดที่สร้างขึ้นในรัชกาลที่ 5 
พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว
มีสถาปัตยกรรมไทยผสมกับกอธิก
มีพระประธานนามว่า พระพุทธอังคีรส
มีสร้อยนามว่า สถิตมหาสีมาราม
Rating: 5
Published Date: 2023-10-21T01:18:50Z
URL: https://www.tripadvisor.com/ShowUserReviews-g293916-d2209612-r922231809-Reviews-Wat_Ratchabophit-Bangkok.html?m=66827#review922231809
--------------------------------------------------
Review ID: 777982512
Username: kant_setpakdee
Review Text: ตั้งอยู่ด้านทิศตะวันตกของวัด มีประตูเข้าตรงทางขึ้นด้านหลังพระเจดีย์ของวัด ภายในบรรจุพระอัฐิ พระสรีรางคาร พระอังคาร และอังคารของ พระมเหสี พระราชชายา เจ้าจอมมารดา และพระราชโอรส พระราชธิดา ในพระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว รัชกาลที่ ๕ เพื่อให้ลูกได้อยู่รวมกับแม่หรืออย่างน้อยในบ้านของแม่ 
Rating: 5
Published Date: 2020-11-27T02:16:31Z
URL: https://www.tripadvisor.com/ShowUserReviews-g293916-d2209612-r777982512-Reviews-Wat_Ratchabophit-Bangkok.html?m=66827#review7

In [6]:
# Define Thai stop words
thai_stopwords = thai_stopwords()

# Function to normalize text
def normalize_text(text):
    tokens = word_tokenize(text, engine='newmm')
    tokens = [token for token in tokens if token not in thai_stopwords and token.isalpha()]
    return ' '.join(tokens)